# Fase 3: Extracción de Detalles de Actores con Selenium - Parte final

Extraer información detallada de los 10 principales actores de cada una de las películas extraidas en la fase 1 utilizando Selenium (desde la página e IMDB). Debereis solo tener en cuenta los 10 primeros actores de cada película o corto (en caso de que los haya)La información que deberás extraer es:

Nombre.

Año nacimiento.

¿Por qué es conocido?

¿Qué hace?

Premios.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. 



In [1]:
import requests
import re

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

from time import sleep

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [29]:

def obtener_detalles_actores(lista_actores,lista_detalles_actores):

    # Crear un diccionario llamado "datos_actor" para almacenar la información de cada uno de los actores. Un diccionario por cada actor.
    datos_actor = {}

    url_actores=(f"https://www.imdb.com/")

    # Inicializa el navegador Chrome
    driver = webdriver.Chrome()

    # Navega a la página web "https://www.imdb.com/"
    driver.get(url_actores)

    driver.maximize_window()

    sleep(2)

    # Aceptar las cookies en la página web

    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

    sleep(3)

    #Vamos a recorrer la lista de actores

    for actor in lista_actores:

            # Ingresar el actor de la lista de actores y coger los datos en el campo de búsqueda y presionar Enter
            
            driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)

            sleep(5)

            # Agrego el valor actor a la clave nombre_actor
            
            datos_actor["nombre_actor"] = actor
            
            # pinchamos en el resultado de la busqueda que nos lleva a la info del actor

            driver.execute_script("window.scrollTo(0,400)")
            
            sleep (2)
            
            driver.find_element('xpath','//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div/a').click()
            
            sleep (3)

            try:
                resultado_que_hace = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
                lista_que_hace = resultado_que_hace.split("\n")
                datos_actor["que_hace"] = lista_que_hace    
            
            except:                
                print("Error al encontrar la seccion 'que hace'")
                datos_actor["que_hace"] = "None"
            
            sleep(2) 

            try:            
                driver.execute_script("window.scrollTo(0,500)")
                anno_nacimiento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text
                resultado_anno_nacimiento = re.search(r'\b\d{4}\b', anno_nacimiento).group()
                datos_actor["anno_nacimiento"] = resultado_anno_nacimiento    
            except:
                print("Error al extrear el año de nacimiento")
                datos_actor['anno_nacimiento'] = "0"

            sleep(3) 
            i = 500
            while i<2000:
                try:
                    driver.execute_script(f"window.scrollTo(0,{i})")
                    sleep(3)
                    
                    try:
                        texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul').text
                        break
                    except:
                        texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul').text
                        break
                except:
                    i+=500
            
            try:
                cantidad_premios = re.search(r'(\d+)\s+premios?',texto_premios, re.IGNORECASE).group(1)
                datos_actor['premios'] = cantidad_premios
        
            except:
                print("Error al extrear los premios")
                datos_actor['premios'] = "0"

            driver.execute_script(f"window.scrollTo({i},0)") 
            
            sleep(4)
            listas_pelis_conocido_por = []
            for x in range(1,5):
                    
                try:
                    driver.execute_script("window.scrollTo(0,2000)")
                    sleep(3)
                    resultado_conocido_por = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-af50cbe8-0.fqoRPX > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({x}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top').text
                    listas_pelis_conocido_por.append(resultado_conocido_por)
                except:                    
                    print("Error al encontrar las peliculas")                    
                    listas_pelis_conocido_por.append("None")

            sleep(2)

            datos_actor["peliculas"] = listas_pelis_conocido_por 
            
            #driver.back()

            try:
                tupla_actor = (datos_actor['nombre_actor'], int(datos_actor['anno_nacimiento']), datos_actor["peliculas"], datos_actor["que_hace"], int(datos_actor['premios']))
                print(type(tupla_actor))
            except:
                print("Error al crear la tupla. Creamos tupla con null")
                tupla_actor = (datos_actor['nombre_actor'], 0, ["None"], ["None"], 0)

            lista_detalles_actores.append(tupla_actor)
                        
            datos_actor = {}
        
    print("----------------")
    
    #print(lista_detalles_actores)

    return(lista_detalles_actores)

In [2]:

def obtener_actores_from_peliculas(lista_id_peliculas, lista_actores):

        url_actores=(f"https://www.imdb.com/")

        # Inicializa el navegador Chrome
        driver = webdriver.Chrome()

        # Navega a la página web "https://www.imdb.com/"
        driver.get(url_actores)

        driver.maximize_window()

        sleep(2)

        # Aceptar las cookies en la página web

        driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

        sleep(2)

        for pelicula in lista_id_peliculas:
                print(pelicula)
                
                try:
                        driver.find_element("css selector", "#suggestion-search").send_keys(pelicula, Keys.ENTER)
                except:
                        print("Pelicula no encontrada")

                driver.execute_script("window.scrollTo(0,500)")
                
                sleep(2)

                for i in range(1,11):

                        try:
                                actor = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})').text.split("\n")[0]                               
                                if actor not in lista_actores:
                                        lista_actores.append(actor)
                                        
                        except:
                                print(f"Error al encontrar los 10 primeros actores de la pelicula {pelicula}")
                sleep(2)                               
                driver.back()
                sleep(2)
                
        return lista_actores

In [26]:
import pandas as pd

csv_url = "https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase1.csv"

df= pd.read_csv(csv_url)
df.columns=['tipo_pelicula','titulo_pelicula','anno_estreno','mes_estreno','id_pelicula','genero_pelicula']
lista_actores = []
lista_id_peliculas = list(df['id_pelicula'])
lista_id_peliculas_1=lista_id_peliculas[0:500]
lista_id_peliculas_2=lista_id_peliculas[500:1001]



In [27]:
len(lista_id_peliculas_2)

500

In [21]:
lista_id_peliculas_1[0]

'tt1929361'

In [24]:
lista_id_peliculas_1[499]

'tt18266556'

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tipo_pelicula    1000 non-null   object
 1   titulo_pelicula  1000 non-null   object
 2   anno_estreno     1000 non-null   int64 
 3   mes_estreno      924 non-null    object
 4   id_pelicula      1000 non-null   object
 5   genero_pelicula  1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [28]:


resultados_lista_actores = obtener_actores_from_peliculas(lista_id_peliculas_2, lista_actores) #obtener lista de actores usando una lista de id peliculas de un CSV

tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar lo

In [29]:
len(lista_actores)


2539

In [30]:
df_lista_actores_2 = pd.DataFrame(lista_actores)

In [31]:
df_lista_actores_2

,0
0,Dan Jopling
1,Croix Provence
2,Aswathi Balan
3,Vijay Karthik
4,Chivvaun Abigial
...,...
2534,Durukan Çelikkaya
2535,Tuana Almaci
2536,Keci Memos
2537,Baris Bayram


In [32]:
df_lista_actores_2.to_csv("lista_actores_2.csv", index=False)

In [19]:
import csv
with open('actores.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerow(lista_actores)

In [31]:
# para que no se pierda la lista obtenida anteriormente:

lista_detalles_actores=[]
resultado_detalles_actores = obtener_detalles_actores(lista_actores,lista_detalles_actores)

Byom
Error al extrear los premios
Error al extrear el año de nacimiento
Rose LaRue
Error al extrear los premios
Error al extrear el año de nacimiento
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar la seccion 'que hace'
Aaron Jackson
Error al extrear los premios
Error al extrear el año de nacimiento
Becky Brown
Error al extrear los premios
Error al extrear el año de nacimiento
Cassidy Reyes
Error al extrear los premios
Error al extrear el año de nacimiento
Mercedes Gutierrez
Error al extrear los premios
Error al extrear el año de nacimiento
Tay Lindsey
Error al extrear los premios
Error al extrear el año de nacimiento
Deborah Smith Ford
Error al crear la tupla. Creamos tupla con null
Jason Menz
Error al extrear los premios
Error al extrear el año de nacimiento
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peli

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (454, 581)
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF790E61572+60802]
	(No symbol) [0x00007FF790DDAC72]
	(No symbol) [0x00007FF790C97CE4]
	(No symbol) [0x00007FF790CEE877]
	(No symbol) [0x00007FF790CEC344]
	(No symbol) [0x00007FF790CE980A]
	(No symbol) [0x00007FF790CE89E2]
	(No symbol) [0x00007FF790CDAE8B]
	(No symbol) [0x00007FF790D0AB7A]
	(No symbol) [0x00007FF790CDA7C6]
	(No symbol) [0x00007FF790D0AD90]
	(No symbol) [0x00007FF790D2A224]
	(No symbol) [0x00007FF790D0A923]
	(No symbol) [0x00007FF790CD8FEC]
	(No symbol) [0x00007FF790CD9C21]
	GetHandleVerifier [0x00007FF79116420D+3217949]
	GetHandleVerifier [0x00007FF7911A61A7+3488183]
	GetHandleVerifier [0x00007FF79119F12F+3459391]
	GetHandleVerifier [0x00007FF790F1B936+823622]
	(No symbol) [0x00007FF790DE600F]
	(No symbol) [0x00007FF790DE0F34]
	(No symbol) [0x00007FF790DE10C2]
	(No symbol) [0x00

In [32]:
import csv
with open('detalles_actores.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerow(lista_detalles_actores)

In [33]:
print(len(lista_actores))
df = pd.DataFrame(lista_detalles_actores)
df

990


,0,1,2,3,4
0,Byom,0,"[Mom's Coming, Lage Na Mon, Love Driver, Jai H...","[Reparto, Producción, Dirección de la segunda ...",0
1,Rose LaRue,0,"[NULL, NULL, NULL, NULL]",NULL,0
2,Aaron Jackson,0,"[Rabos: El musical, The Chris Gethard Show: Pu...","[Reparto, Guion, Producción]",0
3,Becky Brown,0,"[Hanging Millstone, That's What Friends are Fo...","[Reparto, Producción, Casting]",0
4,Cassidy Reyes,0,"[The Bermuda Triangle Project, God Forbid, The...","[Reparto, Producción]",0
5,Mercedes Gutierrez,0,"[Beauty Is Skin Deep, Todd, Crescent City Chro...",[Reparto],0
6,Tay Lindsey,0,"[Step Up Revolution, Time 2 Tell, Quarry, Últi...","[Reparto, Equipos adicionales]",0
7,Deborah Smith Ford,0,[NULL],[NULL],0
8,Jason Menz,0,"[NULL, NULL, NULL, NULL]",NULL,0
9,Miles Christian-Hart,0,"[The 4th Letter of Ponce De Leon, Love Song an...",[Reparto],0
